# <center> 计算图和数据图可视化</center>


## 计算图与数据图概述

计算图的生成是通过将模型训练过程中的每个计算节点关联后所构成的，初体验者可以通过查看计算图，掌握整个模型的计算走向结构，数据流以及控制流的信息。对于高阶的使用人员，能够通过计算图验证计算节点的输入输出是否正确，并验证整个计算过程是否符合预期。数据图展示的是数据预处理的过程，在MindInsight可视化面板中可查看数据处理的图，能够更加直观地查看数据预处理的每一个环节，并帮助提升模型性能。

接下来我们用一个图片分类的项目来体验计算图与数据图的生成与使用。
        
## 本次体验的整体流程
1. 体验模型的数据选择使用MNIST数据集，MNIST数据集整体数据量比较小，更适合体验使用。

2. 初始化一个网络，本次的体验使用LeNet网络。

3. 增加可视化功能的使用，并设定只记录计算图与数据图。

4. 加载训练数据集并进行训练，训练完成后，查看结果并保存模型文件。

5. 启用MindInsight的可视化图界面，进行训练过程的核对。

### 数据集来源

方法一

从以下网址下载，并将数据包解压后放在Jupyter的工作目录下。

- 训练数据集：{"<http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz>","<http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz>"}
- 测试数据集：{"<http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz>","<http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz>"}

可执行下面代码查看Jupyter的工作目录。

In [ ]:
import os
os.getcwd()

- 训练数据集放在----`Jupyter工作目录+\MNIST_Data\train\`，此时`train`文件夹内应该包含两个文件，`train-images-idx3-ubyte`和`train-labels-idx1-ubyte` 
- 测试数据集放在----`Jupyter工作目录+\MNIST_Data\test\`，此时`test`文件夹内应该包含两个文件，`t10k-images-idx3-ubyte`和`t10k-labels-idx1-ubyte`

方法二

直接执行以下代码，会自动进行训练集的下载与解压，但是整个过程根据网络好坏情况会需要花费几分钟时间。

In [ ]:
import os
import gzip
import urllib.request
from urllib.parse import urlparse


def unzip_file(gzip_path):
    """
    Unzip a given gzip file.

    Args:
        gzip_path (str): The gzip file path
    """
    open_file = open(gzip_path.replace('.gz', ''), 'wb')
    gz_file = gzip.GzipFile(gzip_path)
    open_file.write(gz_file.read())
    gz_file.close()


def download_dataset():
    """Download the dataset from http://yann.lecun.com/exdb/mnist/."""
    print("******Downloading the MNIST dataset******")
    train_path = "./MNIST_Data/train/"
    test_path = "./MNIST_Data/test/"
    train_path_check = os.path.exists(train_path)
    test_path_check = os.path.exists(test_path)
    if not train_path_check and not test_path_check:
        os.makedirs(train_path)
        os.makedirs(test_path)
    train_url = {"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
                 "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
    test_url = {"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
                "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}

    for url in train_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(train_path, url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')) and not os.path.exists(file_name):
            urllib.request.urlretrieve(url, file_name)
        unzip_file(file_name)

    for url in test_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(test_path, url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')) and not os.path.exists(file_name):
            urllib.request.urlretrieve(url, file_name)
        unzip_file(file_name)

download_dataset()

#### 数据增强
对数据集进行数据增强操作，可以提升模型精度。


In [ ]:
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.vision import Inter
from mindspore.common import dtype as mstype


def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """
    Create dataset for train or test.

    Args:
        data_path (str): The absolute path of the dataset
        batch_size (int): The number of data records in each group
        repeat_size (int): The number of replicated data records
        num_parallel_workers (int): The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # define some parameters needed for data enhancement and rough justification
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # according to the parameters, generate the corresponding data enhancement method
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # using map method to apply operations to a dataset
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=resize_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_nml_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=hwc2chw_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    
    # process the generated dataset
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

### 可视化操作流程

1. 准备训练脚本，在训练脚本中指定计算图的超参数信息，使用`Summary`保存到日志中，接着再运行训练脚本。

2. 启动MindInsight，启动成功后，就可以通过访问命令执行后显示的地址，查看可视化界面。

3. 访问可视化地址成功后，就可以对图界面进行查询等操作。

#### 初始化网络

1. 导入构建网络所使用的模块。

2. 构建初始化参数的函数。

3. 创建网络，在网络中设置参数。

In [ ]:
import mindspore.nn as nn
from mindspore.common.initializer import TruncatedNormal


def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    """weight initial for conv layer"""
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode="valid")


def fc_with_initialize(input_channels, out_channels):
    """weight initial for fc layer"""
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)


def weight_variable():
    """weight initial"""
    return TruncatedNormal(0.02)


class LeNet5(nn.Cell):
    
    def __init__(self, num_class=10, channel=1):
        super(LeNet5, self).__init__()
        self.num_class = num_class
        self.conv1 = conv(channel, 6, 5)
        self.conv2 = conv(6, 16, 5)
        self.fc1 = fc_with_initialize(16 * 5 * 5, 120)
        self.fc2 = fc_with_initialize(120, 84)
        self.fc3 = fc_with_initialize(84, self.num_class)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

#### 执行训练

1. 导入所需的代码包，并示例化训练网络。
2. 通过MindSpore提供的 `SummaryCollector` 接口，实现收集计算图和数据图。在实例化 `SummaryCollector` 时，在 `collect_specified_data` 参数中，通过设置 `collect_graph` 指定收集计算图，设置 `collect_dataset_graph` 指定收集数据图。

更多 `SummaryCollector` 的用法，请点击[API文档](https://www.mindspore.cn/doc/api_python/zh-CN/r1.0/mindspore/mindspore.train.html#mindspore.train.callback.SummaryCollector)查看。



In [ ]:
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.callback import LossMonitor, SummaryCollector

if __name__ == "__main__":
    device_target = "CPU"
    
    context.set_context(mode=context.GRAPH_MODE, device_target=device_target)
    download_dataset()
    ds_train = create_dataset(data_path="./MNIST_Data/train/")

    network = LeNet5()
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)
    time_cb = TimeMonitor(data_size=ds_train.get_dataset_size())
    model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})

    specified={'collect_graph': True, 'collect_dataset_graph': True}
    summary_collector = SummaryCollector(summary_dir='./summary_dir', collect_specified_data=specified, collect_freq=1, keep_default_action=False)
    
    print("============== Starting Training ==============")
    model.train(epoch=2, train_dataset=ds_train, callbacks=[LossMonitor(), summary_collector], dataset_sink_mode=False)

    print("============== Starting Testing ==============")
    ds_eval = create_dataset("./MNIST_Data/test/")
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("============== {} ==============".format(acc))

### 启动MindInsight
- 启动MindInsigh服务命令：`mindinsigh start --summary-base-dir=/path/ --port=8080`；
- 执行完服务命令后，访问给出的地址，查看MindInsigh可视化结果。

> 其中 /path/ 为 `SummaryCollector` 中参数 `summary_dir` 所指定的目录。

![title](https://gitee.com/mindspore/docs/raw/r1.0/tutorials/notebook/mindinsight/images/mindinsight_map.png)

### 计算图信息
- 文本选择框：输入计算图对应的路径及文件名，显示相应的计算图，便于查找文件。
- 搜索框：可以对整体计算图的节点信息进行搜索，输入完整的节点名称，回车执行搜索，如果有该名称节点，就会呈现出来，便于查找节点。
- 缩略图：展示整体计算图的缩略情况，在面板左边查看详细图结构时，在缩略图处会有定位，显示当前查看的位置在整体计算图中的定位，实时呈现部分与整体的关系。
- 节点信息：显示当前所查看节点的信息，包括名称、类型、属性、输入和输出。便于在训练结束后，核对计算正确性时查看。
- 图例：图例中包括命名空间、聚合节点、虚拟节点、算子节点、常量节点，通过不同图形来区分。

![title](https://gitee.com/mindspore/docs/raw/r1.0/tutorials/notebook/mindinsight/images/cast_map.png)

### 数据图展示

数据图展示了数据增强中对数据进行操作的流程。

1. 首先是从加载数据集 `mnist_ds = ds.MnistDataset(data_path)` 开始，对应数据图中 `MnistDataset`。

2. 下面代码为上面的 `create_dataset` 函数中作数据预处理与数据增强的相关操作。可以从数据图中清晰地看到数据处理的流程。通过查看数据图，可以帮助分析是否存在不恰当的数据处理流程。

```
mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(operations=resize_op, input_columns="image", num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(operations=rescale_op, input_columns="image", num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(operations=rescale_nml_op, input_columns="image", num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(operations=hwc2chw_op, input_columns="image", num_parallel_workers=num_parallel_workers)

mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
mnist_ds = mnist_ds.repeat(repeat_size)
```


![title](https://gitee.com/mindspore/docs/raw/r1.0/tutorials/notebook/mindinsight/images/data_map.png)

### 关闭MindInsight

- 查看完成后，在命令行中可执行此命令 `mindinsight stop --port=8080`，关闭MindInsight。